# Suite database viewer
This notebook was created to capture common commands which may be useful to developers
working on, or helping users with Cylc Suite databases.


In [ ]:
# Set the parameters of the suite whose database you want to investigate
import os
USER = "jbloggs"
SUITENAME = "simplest"
HOMEDIR = os.getenv('HOME')
SUITE_FILE_GLOB_PATTERN = f"{HOMEDIR}/cylc-run/{SUITENAME}/"

# Deliberately left blank before sharing
SQLLITEPATH = ""
print(SUITE_FILE_GLOB_PATTERN)

In [ ]:
# This cell contains boilerplate to ensure that the database exists
from IPython.core.display import HTML, display, Markdown
from glob import glob
    
def printm(text):
    display(Markdown(text))
    
SUITE_FOLDER=glob(SUITE_FILE_GLOB_PATTERN)
if len(SUITE_FOLDER) == 1:
    dbfile=f"{SUITE_FOLDER[0]}log/db"
    printm("This is the datebase file we shall interrogate:")
    !ls -l $dbfile
else:
    msg = (
        f"\nthis username and suite name do not produce a unique answer."
        f"\nThe following files were found:\n"
        f"{SUITE_FOLDER}"
    )
    raise FileNotFoundError(msg)


## SQL introspection
### Tables
Firstly we can get a list of tables available...

In [ ]:
%%script $SQLLITEPATH $dbfile

. tables

### Schema
A good deal more info comes from the schema...

In [ ]:
%%capture N
%%script $SQLLITEPATH  $dbfile

. schema

In [ ]:
for item in N.stdout.split('\n'):
    print(item, '\n')

## Diving into the data
We should now have enough information about the data to start looking in detail at what the database contains:

In [ ]:
%%script $SQLLITEPATH $dbfile

SELECT cycle, name,user_at_host, batch_sys_name from task_jobs 
LIMIT 10

... or, if you want to be really fancy this sort of thing will give you a nicely formatted pandas
table

In [ ]:
%%capture M
%%script $SQLLITEPATH $dbfile

SELECT cycle, name, user_at_host, batch_sys_name from task_jobs 
LIMIT 10

In [ ]:
from io import StringIO
import pandas
pandas.read_csv(
    StringIO(M.stdout),
    sep="|",
    index_col=False,
    names=[
        "cycle", 
        "name", 
        "user_at_host", 
        "batch_sys_name"
    ]
)